
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type" />
<title>Untitled 1</title>
<style type="text/css">
.auto-style1 {
	text-align: left;
}
.auto-style2 {
	text-align: center;
	font-size: x-large;
	font-family: Arial, Helvetica, sans-serif;
}
.auto-style3 {
	font-family: Arial, Helvetica, sans-serif;
}
.auto-style4 {
	font-family: Arial, Helvetica, sans-serif;
	text-decoration: underline;
}
</style>
</head>

<body>

<p class="auto-style2"><strong>Finding Lane Lines on the Road</strong></p>
<p class="auto-style1"><strong><br />
</strong><span class="auto-style3"><strong>Work Scope</strong></span><br class="auto-style3" />
<br class="auto-style3" />
<span class="auto-style3">In this project, you will be writing code to identify 
lane lines on the road, first in an image, and later in a video stream (really 
just a series of images). To complete this project you will use the tools you 
learned about in the lesson, and build upon them.</span></p>


<br>
<br>
<br>
<span class="auto-style4">1 - Dependencies</span><br class="auto-style3">
<span class="auto-style3">Here I inserted all dependencies needed for this project
</span>
</body>

</html>


In [1]:
#Import all dependencies
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math

<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type" />
<title>Project 1: Finding Lane Lines on the Road</title>
<style type="text/css">
.auto-style1 {
	text-align: left;
}
.auto-style2 {
	text-align: center;
	font-size: x-large;
	font-family: Arial, Helvetica, sans-serif;
}
.auto-style3 {
	font-family: Arial, Helvetica, sans-serif;
}
</style>
</head>

<body>


<br class="auto-style3" />
<span class="auto-style3">2.2 - Functions.<br><br>
Here I define a function that will be needed later. This function provides a False or True statement about given point if they are or not included in a polygon.</span></p>

</body>

</html>

In [2]:
def point_in_poly(x,y,poly):

    n = len(poly)
    inside = False

    p1x,p1y = poly[0]
    for i in range(n+1):
        p2x,p2y = poly[i % n]
        if y > min(p1y,p2y):
            if y <= max(p1y,p2y):
                if x <= max(p1x,p2x):
                    if p1y != p2y:
                        xints = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x,p1y = p2x,p2y

    return inside

<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type" />
<title>Project 1: Finding Lane Lines on the Road</title>
<style type="text/css">
.auto-style1 {
	text-align: left;
}
.auto-style2 {
	text-align: center;
	font-size: x-large;
	font-family: Arial, Helvetica, sans-serif;
}
.auto-style3 {
	font-family: Arial, Helvetica, sans-serif;
}
</style>
</head>

<body>


<br class="auto-style3" />
<span class="auto-style3">2.3 - Video Input and Output.<br><br>
Here I use the OpenCV VideoCapture to import the .MP4 video, and use the VideoWriter to create an output file with all frames processed. I tried to generate another MP4 video, but due to my computer Codec issues, I found easier to generate an XVID .avi file.</span></p>

</body>

</html>

In [3]:
#Capture Video Frames
cap = cv2.VideoCapture('solidYellowLeft.mp4')

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('solidYellowLeft_output.avi',fourcc, 20.0, (960,540))

<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type" />
<title>Project 1: Finding Lane Lines on the Road</title>
<style type="text/css">
.auto-style1 {
	text-align: left;
}
.auto-style2 {
	text-align: center;
	font-size: x-large;
	font-family: Arial, Helvetica, sans-serif;
}
.auto-style3 {
	font-family: Arial, Helvetica, sans-serif;
}
</style>
</head>

<body>


<br class="auto-style3" />
<span class="auto-style3">2.4 - Frame to Frame Processing.<br><br>
I used a while loop to process each frame until the source file is open. The frame is saved in a variable with cap.read().
<br><br>
Inside this loop I used basically the algorithms teached in class. First I grayscaled the image using cv2.cvtColor, and then applied the Gaussian Blur using cv2.GaussianBlur.
<br>
With the tunned parameters for low and high threshold defined as 70 and 210, I applied the Canny Filter using cv2.Canny<br><br>I tunned the Hough Transform and the Polygon for mask area following the same as teached in our class. The result is written to lines variable. 
<br>
<br>
After that I run a loop to use the function <i>point_in_poly</i> to check if the points are inside or outside the Polygon defined. Here I'm working with points, diferently from the class where we masked a picture, here I'm masking a matrix of points.
<br>
I also used the same loop to register the maximum and minimum values found for points. These will be usefull to determine the new line between these extreme points.

<br>
<br>
After the loop, I calculate the slope of the line made with the extreme X,Y points. This slope will generate an extrapolate line, that could have the size I want.
<br>
Then I used this extrapolation to determine the crossing points on the lower part of the picture.
<br>
By the end, I define a new line that start in these points that cross with the lower part of the picture. Using the slope and a defined Y (that will determine the lenght of the lines), I generate another point that has exactly this defined Y. These two points generate the final lines.
<br>
The hole processing here is done for the Right and Left side separatelly. I defined the center of the Polygon as the separation between these sides.



</span></p>

</body>

</html>

In [4]:
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        #Apply the grayscale
        gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Define a kernel size for Gaussian smoothing / blurring
        kernel_size = 7 #Should be odd number (1, 3, 5, 7 ...)
        blur_gray = cv2.GaussianBlur(gray_image,(kernel_size, kernel_size), 0)

        #Apply the Canny Filter
        low_threshold = 70
        high_threshold = 210
        edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
        # Next we'll create a masked edges image using cv2.fillPoly()
        mask = np.zeros_like(edges)   
        ignore_mask_color = 255   

        # This time we are defining a four sided polygon to mask
        imshape = frame.shape
        vertices = np.array([[(0,imshape[0]),(450, 320), (490, 320), (imshape[1],imshape[0])]], dtype=np.int32)
        cv2.fillPoly(mask, vertices, ignore_mask_color)
        masked_edges = cv2.bitwise_and(edges, mask)
        
        #Apply the Hough transform 
        rho = 1
        theta = (math.pi/180)
        threshold = 10
        min_line_length = 5
        max_line_gap = 60
        line_image = np.copy(frame)*0 #creating a blank to draw lines on
        lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]), min_line_length, max_line_gap)

        # Iterate over the output "lines" and draw lines on the blank
        #check if the points are inside or outside of the Polygon area
        polygon = [(0,imshape[0]),(460, 320), (490, 320), (imshape[1],imshape[0])]
        x_mean = (450+490)/2 #mean of the polygon
        x_left_max = 0
        x_left_min = imshape[1]
        y_left_max = 0
        y_left_min = imshape[0]

        x_right_max = 0
        x_right_min = imshape[1]
        y_right_max = 0
        y_right_min = imshape[0]
        for line in lines:
            for x1,y1,x2,y2 in line:
                    
                ## Test
                pointone = point_in_poly(x1,y1,polygon)
                pointtwo = point_in_poly(x2,y2,polygon)
                ## Call the function with the points and the polygon
                
                
                if pointone == True and pointtwo == True:
                    cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
                    #get the mean value for X. Take it in two parts, above the mean is right side and under is left side.
                    #Lines to the left
                    if x1 < x_mean :            
                        if y1 > y_left_max:
                            y_left_max = y1
                            x_left_max = x1
                        if y2 > y_left_max:
                            y_left_max = y2
                            x_left_max = x2
                        if y1 < y_left_min:
                            y_left_min = y1
                            x_left_min = x1     
                        if y2 < y_left_min:
                            y_left_min = y2
                            x_left_min = x2         
                            
                    if x1 > x_mean :            
                        if y1 > y_right_max:
                            y_right_max = y1
                            x_right_max = x1
                        if y2 > y_right_max:
                            y_right_max = y2
                            x_right_max = x2
                        if y1 < y_right_min:
                            y_right_min = y1
                            x_right_min = x1        
                        if y2 < y_right_min:
                            y_right_min = y2
                            x_right_min = x2

        #draw the final line between lowest and greater points
        color=[72, 90, 255]
        #BLUE, GREEN, RED
        thickness=10



        #DRAW LINE ON RIGHT SIDE
        #how to define Y0 - Use points x_right_max / y_right_max and extrapolate to Y = 540 (limit)
        slope = ((y_right_max - y_right_min)/(x_right_max-x_right_min))	
        x1 = int(x_right_max + ((imshape[0] - y_right_max)/slope))

        #extrapolate the line using the x1 calculated before and the slope
        y1 = imshape[0] #define it will be start from the bottom of the image
        y = 350  #define the lenght of the lines
        x = int(x1 + ((y - y1)/slope))
        cv2.line(frame, (x1, y1), (x, y), color, thickness)

        #DRAW LINE ON LEFT SIDE
        #how to define Y0 - Use points x_right_max / y_right_max and extrapolate to Y = 540 (limit)
        slope = ((y_left_max - y_left_min)/(x_left_max-x_left_min))	
        x1 = int(x_left_max + ((imshape[0] - y_left_max)/slope))

        #extrapolate the line using the x1 calculated before and the slope
        y1 = imshape[0] #define it will be start from the bottom of the image
        y = 350  #define the lenght of the lines
        x = int(x1 + ((y - y1)/slope))
        cv2.line(frame, (x1, y1), (x, y), color, thickness)


        # write the results
        out.write(frame)

        

    else:
        print("Video Written Sucessfully")
        break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

Video Written Sucessfully




<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type" />
<title>Untitled 1</title>
<style type="text/css">
.auto-style1 {
	text-align: left;
}
.auto-style3 {
	font-family: Arial, Helvetica, sans-serif;
}
.auto-style4 {
	font-family: Arial, Helvetica, sans-serif;
	text-decoration: underline;
}
.auto-style5 {
	text-align: left;
	font-family: Arial, Helvetica, sans-serif;
}
</style>
</head>

<body>

<p class="auto-style1"><strong><br class="auto-style3" />
</strong><span class="auto-style3"><strong>Reflections</strong></span></p>
<p class="auto-style5"><strong>Positive Aspects of this Algorithm:</strong></p>
<p class="auto-style5">- This algorithm can adjust the lower position of the 
screen dynamically. So if the camera position changes, it will adjust itself to 
get new start points.</p>
<p class="auto-style5"><strong>Negative Aspects:</strong></p>
<p class="auto-style5">- This algorithm is pretty much basic. It will not 
perform well when a lot of noisy lines or dots appear inside the Polygon masking 
area. Also it will note cope with changes in color of the tormac.</p>
<p class="auto-style5">- It is computationally costly. I run the code to 
determine the maximum and minimum values manually, and this could cost some CPU 
cycles. I'm used to develop using C for embedded devices, so I'm still learning 
how Python deals with matrix and vectors operations. That why I felt unsecure to 
use these tools and get stuck, rather than going for the basics and get 
something done in time.</p>
<p class="auto-style5">- Some noise and flickering appear on the lines. I tried 
to filter that tunning the parameters without success. I think that there is 
more filter available that can do this work and make them more stable. 
Unfortunatelly I couldnt find anything that worked inside the task deadline.</p>


